In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', 100)

import re
import string
import nltk

from textblob import TextBlob, Word
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# found a major class imbalance after doing work in merge_and_chop, so I'm going to fix that here
# also going to create a csv of roughly 20k instead of doing the train, test, and calidate csv files due to vocabulary problems

#this df is made from merge_and_chop using the dataset files at https://www.kaggle.com/forgemaster/steam-reviews-dataset?select=reviews-1-115.csv
#none of these files are stored in github
df = pd.read_csv("../../../data/combined_split_reviews.csv")
df.head(3)

,review,voted_up
0,A great puzzle game almost anyone can enjoy.It has a cool and humorous storyline I can definantl...,True
1,Nice game !,True
2,good game,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3005689 entries, 0 to 3005688
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   review    object
 1   voted_up  bool  
dtypes: bool(1), object(1)
memory usage: 25.8+ MB


In [4]:
df["voted_up"].value_counts()

True     2613426
False     392263
Name: voted_up, dtype: int64

In [5]:
df["voted_up"].value_counts(normalize=True)

True     0.869493
False    0.130507
Name: voted_up, dtype: float64

In [6]:
# above is where our class imbalance in the train, test, and validate splits comes from
# going to use the two below dataframes to take 10k random voted up and 10k random voted down
df_up = df.loc[df['voted_up'] == 1]
df_down = df.loc[df['voted_up'] == 0]

In [7]:
#getting 10k random voted up
df_up.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2613426 entries, 0 to 3005688
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   review    object
 1   voted_up  bool  
dtypes: bool(1), object(1)
memory usage: 42.4+ MB


In [9]:
X_up = df_up.drop("voted_up", axis=1)
y_up = df_up["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_up, y_up, random_state=57, test_size=0.004) #took some fiddling for test_size
display(X_test.shape, y_test.shape)

(10454, 1)

(10454,)

In [10]:
df_up_sample = pd.concat([X_test, y_test], axis=1)
df_up_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10454 entries, 1966882 to 1081349
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    10454 non-null  object
 1   voted_up  10454 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 173.6+ KB


In [11]:
df_up_sample.head(3)

,review,voted_up
1966882,"This game is fun, but it's way too easy. I couldn't believe how easy the bosses were. They look ...",True
2844582,Really fun and well designed.. would love to see more classes and a coop mode in the future!,True
2853106,Jungle people very angry.,True


In [12]:
#looking good for the up votes, now the downvotes

X_down = df_down.drop("voted_up", axis=1)
y_down = df_down["voted_up"]

X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, random_state=57, test_size=0.027) #took some fiddling for test_size
display(X_test.shape, y_test.shape)

(10592, 1)

(10592,)

In [13]:
df_down_sample = pd.concat([X_test, y_test], axis=1)
df_down_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10592 entries, 2119612 to 1626697
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    10592 non-null  object
 1   voted_up  10592 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 175.8+ KB


In [14]:
df_down_sample.head(3)

,review,voted_up
2119612,You can NOT play this offline. You need a internet connection at all times. Pretty fun sometimes...,False
2537484,"TL;DR, its one of those games that's more fun to watch than it is to play.\r\nyou pick up artifa...",False
1918530,"I simply can not recommend this game at this time. There are some good moments, and so far the o...",False


In [17]:
#now to combine these two and then change voted_up to an int

final_df = pd.concat([df_up_sample, df_down_sample])
final_df.reset_index(drop=True, inplace=True)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21046 entries, 0 to 21045
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    21046 non-null  object
 1   voted_up  21046 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 185.1+ KB


In [18]:
final_df.head(3)

,review,voted_up
0,"This game is fun, but it's way too easy. I couldn't believe how easy the bosses were. They look ...",True
1,Really fun and well designed.. would love to see more classes and a coop mode in the future!,True
2,Jungle people very angry.,True


In [19]:
final_df["voted_up"] = final_df["voted_up"].astype(int)
final_df.head(3)

,review,voted_up
0,"This game is fun, but it's way too easy. I couldn't believe how easy the bosses were. They look ...",1
1,Really fun and well designed.. would love to see more classes and a coop mode in the future!,1
2,Jungle people very angry.,1


In [20]:
final_df["voted_up"].value_counts(normalize=True)

0    0.503279
1    0.496721
Name: voted_up, dtype: float64

In [21]:
final_df.to_csv("../../../data/main.csv", index=False)